### Инструкция по выполнению проекта:

Мы - маркетинговый аналитик развлекательного приложения Procrastinate Pro+. Несколько прошлых месяцев наш бизнес постоянно нес убытки - в привлечение пользователей была вложена куча денег, а толку никакого. Нам нужно разобраться в причинах этой ситуации.

У нас в распоряжении есть лог сервера с данными о посещениях приложения новыми пользователями, зарегистрировавшимися в период с 2019-05-01 по 2019-10-27, выгрузка их покупок за этот период, а также статистика рекламных расходов. Нам предстоит изучить, как люди пользуются продуктом, когда они начинают покупать, сколько денег приносит каждый клиент, когда он окупается и какие факторы отрицательно влияют на привлечение пользователей.

#### Шаг 1. Загрузим данные и подготовим их к анализу
Загрузим данные о визитах, заказах и расходах в переменные. Оптимизируйте данные для анализа. Убедитесь, что тип данных в каждой колонке — правильный.

#### Шаг 2. Зададим функции для расчета и анализа LTV, ROI, удержания и конверсии


#### Шаг 3. Проведем исследовательский анализ данных

Построим профили пользователей. Определим минимальную и максимальную дату привлечения пользователей.

Выясним:
- Из каких стран приходят посетители? Какие страны дают больше всего платящих пользователей?
- Какими устройствами они пользуются? С каких устройств чаще всего заходят платящие пользователи?
- По каким рекламным каналам шло привлечение пользователей? Какие каналы приносят больше всего платящих пользователей?.

#### Шаг 4. Маркетинг
Выясним:

- Сколько денег потратили? Всего / на каждый источник / по времени
- Сколько в среднем стоило привлечение одного покупателя из каждого источника?

#### Шаг 5. Оценим окупаемость рекламы для привлечения пользователей

С помощью LTV и ROI:
- Проанализируем общую окупаемость рекламы;
- Проанализируем окупаемость рекламы с разбивкой по устройствам;
- Проанализируем окупаемость рекламы с разбивкой по странам;
- Проанализируем окупаемость рекламы с разбивкой по рекламным каналам.

Опишем проблемы, которые мы обнаружили. Ответим на вопросы:
- Окупается ли реклама, направленная на привлечение пользователей в целом? 
- Какие устройства, страны и рекламные каналы могут оказывать негативное влияние на окупаемость рекламы?
- Чем могут быть вызваны проблемы окупаемости? Изучите конверсию и удержание с разбивкой по устройствам, странам, рекламным каналам.

Опишем возможные причины обнаруженных проблем и сформируйте рекомендации для рекламного отдела. При решении этого шага считаем, что мы смотрим данные 1-го ноября 2019 года и что в нашей организации принято считать, что окупаемость должна наступать не позднее, чем через 2 недели после привлечения пользователей.

Подумаем, нужно ли включать в анализ органических пользователей?

#### Шаг 6. Напишем выводы
- Выделим причины неэффективности привлечения пользователей;
- Сформируем рекомендации для отдела маркетинга для повышения эффективности.


#### Описание данных
Таблица visits_log_short (лог сервера с информацией о посещениях сайта):

    User Id — уникальный идентификатор пользователя
    Device — категория устройства пользователя
    Session start — дата и время начала сессии
    Session End — дата и время окончания сессии
    Channel — идентификатор рекламного источника, из которого пришел пользователь
    Region - страна пользователя

Таблица orders_log_short (информация о заказах):

    User Id — уникальный id пользователя, который сделал заказ
    Event Dt — дата и время покупки
    Revenue — выручка

Таблица costs_short (информация о затратах на маркетинг):

    Channel — идентификатор рекламного источника
    Dt — дата
    Costs — затраты на этот рекламный источник в этот день

# Шаг 1. Загрузим данные и подготовим их к анализу
Загрузим данные о визитах, заказах и расходах в переменные. Оптимизируем данные для анализа. Убедимся, что тип данных в каждой колонке — правильный. 


In [1]:
#Импортируем библиотеки
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
from scipy import stats as st
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')


In [ ]:
#Изучим таблицу visits_log_short (лог сервера с информацией о посещениях сайта): 
try:
    visits = pd.read_csv('C:/Users/datasets/visits_info_short.csv')  # Локальный путь
except:
    visits = pd.read_csv('/datasets/visits_info_short.csv') # Серверный путь 
visits.info() #общая информация о данных
print(visits.head(10)) #вывели на экран 10 первых значений, чтобы ознакомиться с таблицей
for col in visits.columns:
    print("-----------------------------")
    print(col)
    display(visits[col].describe(include=None))
    display(visits[col].value_counts()) #оценим значения данных с помощью функци describe и value_counts

print(visits.isna().sum()*100/len(visits)) #посмотрим на процент пропусков во всех колонках одновременно
print('Количество дубликатов', visits.duplicated().sum()) #проверяем количество дубликатов

In [ ]:
#Изучим orders_log_short (информация о заказах): 
try:
    orders = pd.read_csv('C:/Users/datasets/orders_info_short.csv')  # Локальный путь
except:
    orders = pd.read_csv('/datasets/orders_info_short.csv') # Серверный путь 
orders.info() #общая информация о данных
print(orders.head(10)) #вывели на экран 10 первых значений, чтобы ознакомиться с таблицей
for col in orders.columns:
    print("-----------------------------")
    print(col)
    display(orders[col].describe(include=None))
    display(orders[col].value_counts()) #оценим значения данных с помощью функци describe и value_counts

print(orders.isna().sum()*100/len(orders)) #посмотрим на процент пропусков во всех колонках одновременно
print('Количество дубликатов', orders.duplicated().sum()) #проверяем количество дубликатов

In [ ]:
#Изучим таблицу costs_short (информация о затратах на маркетинг): 
try:
    costs = pd.read_csv('C:/Users/datasets/costs_info_short.csv')  # Локальный путь
except:
    costs = pd.read_csv('/datasets/costs_info_short.csv') # Серверный путь 
costs.info() #общая информация о данных
print(costs.head(10)) #вывели на экран 10 первых значений, чтобы ознакомиться с таблицей
for col in costs.columns:
    print("-----------------------------")
    print(col)
    display(costs[col].describe(include=None))
    display(costs[col].value_counts()) #оценим значения данных с помощью функци describe и value_counts

print(costs.isna().sum()*100/len(costs)) #посмотрим на процент пропусков во всех колонках одновременно
print('Количество дубликатов', costs.duplicated().sum()) #проверяем количество дубликатов

# Вывод
* По данным таблиц видно, что в них нет пропущенных значений и дубликатов.
* В следующем шаге необходимо будет преобразовать данные о времени.
* Из таблицы о визитах можно выделить 4 страны пользователя:
 - United States    207327
 - UK               
 - France           
 - Germany
* 4 категории  устройста:
 - iPhone     
 - Android     
 - PC          
 - Mac   
* 11 рекламных источников: 
 - organic               
 - TipTop                 
 - FaceBoom               
 - WahooNetBanner         
 - LeapBob             
 - OppleCreativeMedia    
 - RocketSuperAds  
 - YRabbit                
 - MediaTornado            
 - AdNonSense              
 - lambdaMediaAds   



In [ ]:
# Для удобства приведем все названия столбцов к нижнему регистру и уберём пробелы в названиях столбцов
visits.columns = visits.columns.str.lower()
orders.columns = orders.columns.str.lower()
costs.columns = costs.columns.str.lower()

visits.rename(columns = {'user id' : 'user_id', 'session start' : 'session_start', 'session end' : 'session_end'}, inplace = True)
orders.rename(columns = {'user id' : 'user_id', 'event dt' : 'event_dt'}, inplace = True)

# преобразование данных о времени
visits['session_start'] = pd.to_datetime(visits['session_start'])
visits['session_end'] = pd.to_datetime(visits['session_end'])
orders['event_dt'] = pd.to_datetime(orders['event_dt'])
costs['dt'] = pd.to_datetime(costs['dt']).dt.date 

# Посмотрим на 5 первыхстрок наших таблиц, проверим внесённые изменения
display(visits.head(5))
display(orders.head(5))
display(costs.head(5))

# Шаг 2. Зададим функции для расчета и анализа LTV, ROI, удержания и конверсии



In [ ]:
# функция для создания пользовательских профилей

def get_profiles(sessions, orders, ad_costs, event_names=[]):

    # находим параметры первых посещений
    profiles = (
        sessions.sort_values(by=['user_id', 'session_start'])
        .groupby('user_id')
        .agg(
            {
                'session_start': 'first',
                'channel': 'first',
                'device': 'first',
                'region': 'first',
            }
        )
        .rename(columns={'session_start': 'first_ts'})
        .reset_index()
    )

    # для когортного анализа определяем дату первого посещения
    # и первый день месяца, в который это посещение произошло
    profiles['dt'] = profiles['first_ts'].dt.date
    profiles['month'] = profiles['first_ts'].astype('datetime64[M]')

    # добавляем признак платящих пользователей
    profiles['payer'] = profiles['user_id'].isin(orders['user_id'].unique())


    # считаем количество уникальных пользователей
    # с одинаковыми источником и датой привлечения
    new_users = (
        profiles.groupby(['dt', 'channel'])
        .agg({'user_id': 'nunique'})
        .rename(columns={'user_id': 'unique_users'})
        .reset_index()
    )

    # объединяем траты на рекламу и число привлечённых пользователей
    ad_costs = ad_costs.merge(new_users, on=['dt', 'channel'], how='left')

    # делим рекламные расходы на число привлечённых пользователей
    ad_costs['acquisition_cost'] = ad_costs['costs'] / ad_costs['unique_users']

    # добавляем стоимость привлечения в профили
    profiles = profiles.merge(
        ad_costs[['dt', 'channel', 'acquisition_cost']],
        on=['dt', 'channel'],
        how='left',
    )

    # стоимость привлечения органических пользователей равна нулю
    profiles['acquisition_cost'] = profiles['acquisition_cost'].fillna(0)

    return profiles


In [7]:
# функция для расчёта удержания

def get_retention(
    profiles,
    sessions,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # добавляем столбец payer в передаваемый dimensions список
    dimensions = ['payer'] + dimensions

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    # собираем «сырые» данные для расчёта удержания
    result_raw = result_raw.merge(
        sessions[['user_id', 'session_start']], on='user_id', how='left'
    )
    result_raw['lifetime'] = (
        result_raw['session_start'] - result_raw['first_ts']
    ).dt.days

    # функция для группировки таблицы по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        result = df.pivot_table(
            index=dims, columns='lifetime', values='user_id', aggfunc='nunique'
        )
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        result = result.div(result['cohort_size'], axis=0)
        result = result[['cohort_size'] + list(range(horizon_days))]
        result['cohort_size'] = cohort_sizes
        return result

    # получаем таблицу удержания
    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)

    # получаем таблицу динамики удержания
    result_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    # возвращаем обе таблицы и сырые данные
    return result_raw, result_grouped, result_in_time

In [8]:

# функция для расчёта конверсии

def get_conversion(
    profiles,
    purchases,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    # определяем дату и время первой покупки для каждого пользователя
    first_purchases = (
        purchases.sort_values(by=['user_id', 'event_dt'])
        .groupby('user_id')
        .agg({'event_dt': 'first'})
        .reset_index()
    )

    # добавляем данные о покупках в профили
    result_raw = result_raw.merge(
        first_purchases[['user_id', 'event_dt']], on='user_id', how='left'
    )

    # рассчитываем лайфтайм для каждой покупки
    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days

    # группируем по cohort, если в dimensions ничего нет
    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users' 
        dimensions = dimensions + ['cohort']

    # функция для группировки таблицы по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        result = df.pivot_table(
            index=dims, columns='lifetime', values='user_id', aggfunc='nunique'
        )
        result = result.fillna(0).cumsum(axis = 1)
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        # делим каждую «ячейку» в строке на размер когорты
        # и получаем conversion rate
        result = result.div(result['cohort_size'], axis=0)
        result = result[['cohort_size'] + list(range(horizon_days))]
        result['cohort_size'] = cohort_sizes
        return result

    # получаем таблицу конверсии
    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)

    # для таблицы динамики конверсии убираем 'cohort' из dimensions
    if 'cohort' in dimensions: 
        dimensions = []

    # получаем таблицу динамики конверсии
    result_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    # возвращаем обе таблицы и сырые данные
    return result_raw, result_grouped, result_in_time

In [9]:
# функция для расчёта LTV и ROI

def get_ltv(
    profiles,
    purchases,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')
    # добавляем данные о покупках в профили
    result_raw = result_raw.merge(
        purchases[['user_id', 'event_dt', 'revenue']], on='user_id', how='left'
    )
    # рассчитываем лайфтайм пользователя для каждой покупки
    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days
    # группируем по cohort, если в dimensions ничего нет
    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users'
        dimensions = dimensions + ['cohort']

    # функция группировки по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        # строим «треугольную» таблицу выручки
        result = df.pivot_table(
            index=dims, columns='lifetime', values='revenue', aggfunc='sum'
        )
        # находим сумму выручки с накоплением
        result = result.fillna(0).cumsum(axis=1)
        # вычисляем размеры когорт
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        # объединяем размеры когорт и таблицу выручки
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        # считаем LTV: делим каждую «ячейку» в строке на размер когорты
        result = result.div(result['cohort_size'], axis=0)
        # исключаем все лайфтаймы, превышающие горизонт анализа
        result = result[['cohort_size'] + list(range(horizon_days))]
        # восстанавливаем размеры когорт
        result['cohort_size'] = cohort_sizes

        # собираем датафрейм с данными пользователей и значениями CAC, 
        # добавляя параметры из dimensions
        cac = df[['user_id', 'acquisition_cost'] + dims].drop_duplicates()

        # считаем средний CAC по параметрам из dimensions
        cac = (
            cac.groupby(dims)
            .agg({'acquisition_cost': 'mean'})
            .rename(columns={'acquisition_cost': 'cac'})
        )

        # считаем ROI: делим LTV на CAC
        roi = result.div(cac['cac'], axis=0)

        # удаляем строки с бесконечным ROI
        roi = roi[~roi['cohort_size'].isin([np.inf])]

        # восстанавливаем размеры когорт в таблице ROI
        roi['cohort_size'] = cohort_sizes

        # добавляем CAC в таблицу ROI
        roi['cac'] = cac['cac']

        # в финальной таблице оставляем размеры когорт, CAC
        # и ROI в лайфтаймы, не превышающие горизонт анализа
        roi = roi[['cohort_size', 'cac'] + list(range(horizon_days))]

        # возвращаем таблицы LTV и ROI
        return result, roi

    # получаем таблицы LTV и ROI
    result_grouped, roi_grouped = group_by_dimensions(
        result_raw, dimensions, horizon_days
    )

    # для таблиц динамики убираем 'cohort' из dimensions
    if 'cohort' in dimensions:
        dimensions = []

    # получаем таблицы динамики LTV и ROI
    result_in_time, roi_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    return (
        result_raw,  # сырые данные
        result_grouped,  # таблица LTV
        result_in_time,  # таблица динамики LTV
        roi_grouped,  # таблица ROI
        roi_in_time,  # таблица динамики ROI
    )

In [10]:
# функция для сглаживания фрейма

def filter_data(df, window):
    # для каждого столбца применяем скользящее среднее
    for column in df.columns.values:
        df[column] = df[column].rolling(window).mean() 
    return df

In [11]:
# функция для визуализации удержания

def plot_retention(retention, retention_history, horizon, window=7):

    # задаём размер сетки для графиков
    plt.figure(figsize=(15, 10))

    # исключаем размеры когорт и удержание первого дня
    retention = retention.drop(columns=['cohort_size', 0])
    # в таблице динамики оставляем только нужный лайфтайм
    retention_history = retention_history.drop(columns=['cohort_size'])[
        [horizon - 1]
    ]

    # если в индексах таблицы удержания только payer,
    # добавляем второй признак — cohort
    if retention.index.nlevels == 1:
        retention['cohort'] = 'All users'
        retention = retention.reset_index().set_index(['cohort', 'payer'])

    # в таблице графиков — два столбца и две строки, четыре ячейки
    # в первой строим кривые удержания платящих пользователей
    ax1 = plt.subplot(2, 2, 1)
    retention.query('payer == True').droplevel('payer').T.plot(
        grid=True, ax=ax1
    )
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('Удержание платящих пользователей')

    # во второй ячейке строим кривые удержания неплатящих
    # вертикальная ось — от графика из первой ячейки
    ax2 = plt.subplot(2, 2, 2, sharey=ax1)
    retention.query('payer == False').droplevel('payer').T.plot(
        grid=True, ax=ax2
    )
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('Удержание неплатящих пользователей')

    # в третьей ячейке — динамика удержания платящих
    ax3 = plt.subplot(2, 2, 3)
    # получаем названия столбцов для сводной таблицы
    columns = [
        name
        for name in retention_history.index.names
        if name not in ['dt', 'payer']
    ]
    # фильтруем данные и строим график
    filtered_data = retention_history.query('payer == True').pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax3)
    plt.xlabel('Дата привлечения')
    plt.title(
        'Динамика удержания платящих пользователей на {}-й день'.format(
            horizon
        )
    )

    # в чётвертой ячейке — динамика удержания неплатящих
    ax4 = plt.subplot(2, 2, 4, sharey=ax3)
    # фильтруем данные и строим график
    filtered_data = retention_history.query('payer == False').pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax4)
    plt.xlabel('Дата привлечения')
    plt.title(
        'Динамика удержания неплатящих пользователей на {}-й день'.format(
            horizon
        )
    )
    
    plt.tight_layout()
    plt.show()

In [12]:
# функция для визуализации конверсии

def plot_conversion(conversion, conversion_history, horizon, window=7):

    # задаём размер сетки для графиков
    plt.figure(figsize=(15, 5))

    # исключаем размеры когорт
    conversion = conversion.drop(columns=['cohort_size'])
    # в таблице динамики оставляем только нужный лайфтайм
    conversion_history = conversion_history.drop(columns=['cohort_size'])[
        [horizon - 1]
    ]

    # первый график — кривые конверсии
    ax1 = plt.subplot(1, 2, 1)
    conversion.T.plot(grid=True, ax=ax1)
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('Конверсия пользователей')

    # второй график — динамика конверсии
    ax2 = plt.subplot(1, 2, 2, sharey=ax1)
    columns = [
        # столбцами сводной таблицы станут все столбцы индекса, кроме даты
        name for name in conversion_history.index.names if name not in ['dt']
    ]
    filtered_data = conversion_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax2)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика конверсии пользователей на {}-й день'.format(horizon))

    plt.tight_layout()
    plt.show()

In [13]:
# функция для визуализации LTV и ROI

def plot_ltv_roi(ltv, ltv_history, roi, roi_history, horizon, window=7):

    # задаём сетку отрисовки графиков
    plt.figure(figsize=(20, 10))

    # из таблицы ltv исключаем размеры когорт
    ltv = ltv.drop(columns=['cohort_size'])
    # в таблице динамики ltv оставляем только нужный лайфтайм
    ltv_history = ltv_history.drop(columns=['cohort_size'])[[horizon - 1]]

    # стоимость привлечения запишем в отдельный фрейм
    cac_history = roi_history[['cac']]

    # из таблицы roi исключаем размеры когорт и cac
    roi = roi.drop(columns=['cohort_size', 'cac'])
    # в таблице динамики roi оставляем только нужный лайфтайм
    roi_history = roi_history.drop(columns=['cohort_size', 'cac'])[
        [horizon - 1]
    ]

    # первый график — кривые ltv
    ax1 = plt.subplot(2, 3, 1)
    ltv.T.plot(grid=True, ax=ax1)
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('LTV')

    # второй график — динамика ltv
    ax2 = plt.subplot(2, 3, 2, sharey=ax1)
    # столбцами сводной таблицы станут все столбцы индекса, кроме даты
    columns = [name for name in ltv_history.index.names if name not in ['dt']]
    filtered_data = ltv_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax2)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика LTV пользователей на {}-й день'.format(horizon))

    # третий график — динамика cac
    ax3 = plt.subplot(2, 3, 3, sharey=ax1)
    # столбцами сводной таблицы станут все столбцы индекса, кроме даты
    columns = [name for name in cac_history.index.names if name not in ['dt']]
    filtered_data = cac_history.pivot_table(
        index='dt', columns=columns, values='cac', aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax3)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика стоимости привлечения пользователей')

    # четвёртый график — кривые roi
    ax4 = plt.subplot(2, 3, 4)
    roi.T.plot(grid=True, ax=ax4)
    plt.axhline(y=1, color='red', linestyle='--', label='Уровень окупаемости')
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('ROI')

    # пятый график — динамика roi
    ax5 = plt.subplot(2, 3, 5, sharey=ax4)
    # столбцами сводной таблицы станут все столбцы индекса, кроме даты
    columns = [name for name in roi_history.index.names if name not in ['dt']]
    filtered_data = roi_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax5)
    plt.axhline(y=1, color='red', linestyle='--', label='Уровень окупаемости')
    plt.xlabel('Дата привлечения')
    plt.title('Динамика ROI пользователей на {}-й день'.format(horizon))

    plt.tight_layout()
    plt.show()

# Вывод 
На втором шаге мы обозначили функции для расчета и анализа LTV, ROI, удержания и конверсии.

# Шаг 3. Проведем исследовательский анализ данных

Построим профили пользователей. Определим минимальную и максимальную дату привлечения пользователей.

Выясним:
- Из каких стран приходят посетители? Какие страны дают больше всего платящих пользователей?
- Какими устройствами они пользуются? С каких устройств чаще всего заходят платящие пользователи?
- По каким рекламным каналам шло привлечение пользователей? Какие каналы приносят больше всего платящих пользователей?

In [ ]:
# получаем профили пользователей
profiles = get_profiles(visits, orders, costs)

display(profiles.head(5)) 

#Определим минимальную и максимальную дату привлечения пользователей
min_date = profiles['dt'].min() 
max_date = profiles['dt'].max() 
print('Минимальная дата привлечения пользователей:', min_date)
print('Максимальная дата привлечения пользователей:', max_date)

In [ ]:
# посчитаем, из каких стран приходят посетители и их кооличество.
#display(
#    profiles.groupby('region').agg({'user_id': 'nunique'}).sort_values('user_id', ascending=False)
#)

# посчитаем, какая доля новых посетителей стала покупателями в каждой стране.
#display(
#    profiles.groupby('region').agg({'payer': 'mean'}).sort_values('payer', ascending=False)
#)

region_users = profiles.groupby('region').agg({'user_id': 'nunique', 'payer': ['sum', 'mean']}).sort_values(by=[('payer', 'sum')],  ascending = False)
region_users.columns = ['Пользователи', 'Платящие пользователи', '% платящих']
region_users['% платящих']= round((region_users['% платящих']*100), 2)

display(region_users)

In [ ]:
# посчитаем, какими устройствами пользуются посетители. 

#display(
#    profiles.groupby('device').agg({'user_id': 'nunique'}).sort_values('user_id', ascending=False)
#)

# посмотрим, с каких устройств чаще всего заходят платящие пользователи.
#display(
#    profiles.groupby('device').agg({'payer': 'mean'}).sort_values('payer', ascending=False)
#)
device_users = profiles.groupby('device').agg({'user_id': 'nunique', 'payer': ['sum', 'mean']}).sort_values(by=[('payer', 'sum')],  ascending = False)
device_users.columns = ['Пользователи', 'Платящие пользователи', '% платящих']
device_users['% платящих']= round((device_users['% платящих']*100), 2)

display(device_users)

In [ ]:
# посчитаем, по каким рекламным каналам шло привлечение пользователей.
#display(
 #   profiles.groupby('channel').agg({'user_id': 'nunique'}).sort_values('user_id', ascending=False)
#)

# посмотрим, какие каналы приносят больше всего платящих пользователей.
#display(
 #   profiles.groupby('channel').agg({'payer': 'mean'}).sort_values('payer', ascending=False)
#)

channel_users = profiles.groupby('channel').agg({'user_id': 'nunique', 'payer': ['sum', 'mean']}).sort_values(by=[('payer', 'sum')],  ascending = False)
channel_users.columns = [ 'Пользователи', 'Платящие пользователи', '% платящих']
channel_users['% платящих']= round((channel_users['% платящих']*100), 2)

channel_users['Платящие пользователи']= round(channel_users['Платящие пользователи'])

display(channel_users)


# Вывод
* На данном шаге мы получили профили пользователей, нашли минимальную дату привелечения пользователей 01.05.2019г. и  максимальную - 27.10.2019г.
* Выяснили, что больше всего посетителей из США (100002). Количество поситителей из Франции (17450) и Великобритании (17575) практически одинаковое, а вот из Германии меньше всего (14981). А вот платящих пользователей больше всего в США, а вот на втором месте Германия, не смотря на то, что количество поситителей из этой страны наименьшее.
* Самое популярное утсройство iPhone, на втором месте - Android, PC и MAC делят 3 и 4 место (количество пользователей данных устройств практически одинаковое). Платящие пользователи предпочитают заходить с MAC, на втором месте - iPhone, на 3 и 4 соответсвенно Android и PC. По этим данным мы видим, что пользователи MAC платят охотнее, чем пользователи других устройств.
* Большевсего пользователей пришло без рекламы (organic). Каналы FaceBoom и TipTop привлекают больше пользователей, а lambdaMediaAds - меньше всего. Топ-3 канала с наибольшей долей платящих пользователей: 
 - FaceBoom	
 - AdNonSense	
 - lambdaMediaAds	

# Шаг 4. Маркетинг

Выясним:
- Сколько денег потратили? Всего / на каждый источник / по времени
- Сколько в среднем стоило привлечение одного покупателя из каждого источника?

In [ ]:
#Посчитаем, сколько всего денег было потрачено на рекламу:
print('Всего денег потрачено: ', round(profiles['acquisition_cost'].sum(), 2))
display(
    profiles.groupby('channel').agg({'acquisition_cost': 'sum'}).sort_values('acquisition_cost', ascending=False)
)

# Посмотрим зависимость трат на рекламу по каналыам
profiles.pivot_table(index = ['dt'], columns = 'channel', values = 'acquisition_cost', aggfunc = 'sum').plot( grid=True,  figsize = (20,15), linewidth = 3)
plt.title('Изменение метрик по времени')
plt.ylabel('sum_acquisition_cost')
plt.show()


profiles_sort=profiles.groupby('channel').agg({'acquisition_cost': 'sum', 'user_id': 'nunique'}).sort_values('acquisition_cost', ascending=False)
profiles_sort['cost_one']=profiles_sort['acquisition_cost']/profiles_sort['user_id']
display(profiles_sort)


print('Общий средний САС по всему проекту:', round(profiles_sort['acquisition_cost'].mean(),2))
profiles.pivot_table(index = ['month'], values = 'acquisition_cost', aggfunc = 'sum').plot( grid=True,  figsize = (20,15), linewidth = 3)
plt.title('Динамика общих маркетинговых затрат по месяцам')
plt.ylabel('sum_acquisition_cost')
plt.show()

# Посмотрим зависимость трат на рекламу по каналыам
profiles.pivot_table(index = ['month'], columns = 'channel', values = 'acquisition_cost', aggfunc = 'sum').plot( grid=True,  figsize = (20,15), linewidth = 3)
plt.title('Изменение метрик по месяцам')
plt.ylabel('sum_acquisition_cost')
plt.show()


    

# Вывод
* По полученным данным видно, что всего денег на маркетинг потратили 105497.3 ден.единиц.
* Больше всего вложили денег в источники TipTop и FaceBoom, меньше всего в YRabbit. В organic соответственно 0, так как это пользователи 'пришедшие' без рекламы. 
* По графику также видно, что за все время исследования было больше всего вложено в источники TipTop и FaceBoom.
* В нижней таблице в столбце cost_one видим среднюю стоимость привлечения одного покупателя из каждого источника.





# Шаг 5. Оценим окупаемость рекламы для привлечения пользователей

С помощью LTV и ROI:
- Проанализируем общую окупаемость рекламы;
- Проанализируем окупаемость рекламы с разбивкой по устройствам;
- Проанализируем окупаемость рекламы с разбивкой по странам;
- Проанализируем окупаемость рекламы с разбивкой по рекламным каналам.

Опишем проблемы, которые мы обнаружили. Ответим на вопросы:
- Окупается ли реклама, направленная на привлечение пользователей в целом? 
- Какие устройства, страны и рекламные каналы могут оказывать негативное влияние на окупаемость рекламы?
- Чем могут быть вызваны проблемы окупаемости? Изучите конверсию и удержание с разбивкой по устройствам, странам, рекламным каналам.

Опишем возможные причины обнаруженных проблем и сформируем рекомендации для рекламного отдела. При решении этого шага считаем, что мы смотрим данные 1-го ноября 2019 года и что в вашей организации принято считать, что окупаемость должна наступать не позднее, чем через 2 недели после привлечения пользователей.

### Проанализируем общую окупаемость рекламы

In [19]:
observation_date = datetime(2019, 11, 1).date()  # момент анализа
horizon_days = 14  # горизонт анализа 
profiles = profiles.query('channel != "organic"') # Исключим органических пользователей

In [ ]:
#Для начала оценим общую ситуацию — посмотрим на окупаемость рекламы. Рассчитаем и визуализируем LTV и ROI, вызвав функции get_ltv() и plot_ltv_roi().
# считаем LTV и ROI
ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles, orders, observation_date, horizon_days
)

# строим графики
plot_ltv_roi(ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days) 

# Вывод
По графикам можно сделать такие выводы:
* Реклама не окупается. ROI в конце 2 недель не превышает 80%.
* CAC увеличивается. Т.е. происходит увеличение рекламного бюджета. Особенно виден скачок в июне.
* Динамика ROI пользователей на 14-йдень уменьшается для пользователей, которые пришли после середины июля 2019г.
* LTV не превышает 0.9 ден.ед.

### Проанализирем окупаемость рекламы с разбивкой по устройствам

In [ ]:
# Начнём с разбивки по устройствам: передадим параметру dimensions столбец device.
# смотрим окупаемость с разбивкой по устройствам
dimensions = ['device']

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=14
) 

# Вывод
Пользователи PC окупаются, а вот остальные — нет. И чем позже пришли пользователи - тем хуже они 'окупаются'. Так же видно, что стоимость привлечения клиентов у пользователей iPhone и Mac выше остальных.

In [ ]:
# Посмотрим конверсию с разбивкой по устройствам

conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles, orders, observation_date, horizon_days, dimensions=['device']
)

plot_conversion(conversion_grouped, conversion_history, horizon_days) 

# Вывод
Судя по графикам, пользователи PC конвертируются хуже всего.


In [ ]:
# смотрим удержание с разбивкой по устройствам

retention_raw, retention_grouped, retention_history = get_retention(
    profiles, visits, observation_date, horizon_days, dimensions=['device']
)

plot_retention(retention_grouped, retention_history, horizon_days) 


# Вывод
А на этом графике видно, что платящие пользователи всех устройств удерживаются примерно одинаково, PC чуть лучше. Скорей всего проблема с окупаемостью не в устройстве. Но проверить ошибки на устроствах iPhone и Mac все же стоит.

### Проанализируем окупаемость рекламы с разбивкой по странам


In [ ]:
# Разбивка по странам: передадим параметру dimensions столбец region.
# смотрим окупаемость с разбивкой по странам
dimensions = ['region']

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=14
) 

Вот что говорят графики:
* Реклама не окупается только в USA.
* Стоимость привлечения стабильна и примерно одинакова для всех стран, кроме USA.
* LTV для пользователей из USA больше, чем для пользователей из других стран.
* Хуже всего окупается USA.

Возможно, все-таки дело в стране — эту версию не нужно исключать.

In [ ]:
# Посмотрим конверсию с разбивкой по странам

conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles, orders, observation_date, horizon_days, dimensions=['region']
)

plot_conversion(conversion_grouped, conversion_history, horizon_days) 

# Вывод
Судя по графикам, пользователи из USA конвертируются лучше остальных

In [ ]:
# смотрим удержание с разбивкой по странам

retention_raw, retention_grouped, retention_history = get_retention(
    profiles, visits, observation_date, horizon_days, dimensions=['region']
)

plot_retention(retention_grouped, retention_history, horizon_days) 

# Вывод
Действительно, пользователи из USA стабильно плохо удерживаются. Для платящих пользователей из USA удержание 14-го дня ниже, чем для пользователей из других стран.

### Проанализируем окупаемость рекламы с разбивкой по рекламным каналам

In [ ]:
# Разбивка по каналам: передадим параметру dimensions столбец channel.
# смотрим окупаемость с разбивкой по каналам
dimensions = ['channel']

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=14
) 

# Вывод
Стоимость привлечения пользователей в TipTop не стабильна в отличае от других каналов, сильно увеличивается со временем, окупаемость у большинства каналов неплохая. 
И пользователи  приходящие из канала TipTop совсем не окупаются, стоит задуматься об использовании данного канала. На втором месте по стоимости привлечения каналов пользователи пришедшие из FaceBoom, но там все стабильно, стоимость не меняется.

In [ ]:
# смотрим конверсию с разбивкой по каналам

conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles, orders, observation_date, horizon_days, dimensions=['channel']
)

plot_conversion(conversion_grouped, conversion_history, horizon_days) 


# Вывод
Пользователи из канала FaceBoom конвертируются лучше остальных

In [ ]:
# смотрим удержание с разбивкой по каналам

retention_raw, retention_grouped, retention_history = get_retention(
    profiles, visits, observation_date, horizon_days, dimensions=['channel']
)

plot_retention(retention_grouped, retention_history, horizon_days) 


# Вывод
А вот удержание платящих пользователей, пришедших из каналов FaceBoom и AdNonSense хуже всех. 

In [ ]:
# отфильтруем данные по США и построим небольшой отчет по конкретно каналам, действующим в США, затем оценить для них конверсию и удержание 
profiles_problem = profiles.query('region == "United States"') 

# Построим ROI, LTV, CAC
dimensions = ['channel']

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles_problem, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=14
) 



# Вывод

Графики по каналам США подтверждают, что TipTop показывает наихудшую статистику и проблема кроется скорей всего здесь.

In [ ]:
# Оценим конверсию и удержание по каналам США
conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles_problem, orders, observation_date, horizon_days, dimensions=['channel']
)

plot_conversion(conversion_grouped, conversion_history, horizon_days) 




retention_raw, retention_grouped, retention_history = get_retention(
    profiles_problem, visits, observation_date, horizon_days, dimensions=['channel']
)

plot_retention(retention_grouped, retention_history, horizon_days) 


# Вывод
А по этим графикам мы видим, что хоть конверсия по каналу FaceBoom самая высокая, но вот удержания данных пользователей плохое.

# Шаг 6. Напишем выводы
Мы взяли данные о посещениях приложения новыми пользователями, зарегистрировавшимися в период с 2019-05-01 по 2019-10-27. Момент анализа -1.11.2019, а горизонт анализа составил 14 дней.

После нашшего исследования мы видим, что реклама не окупается.
Таккже стоимость привлечения пользователей в TipTop увеличивается,но пользователи  приходящие из канала TipTop совсем не окупаются. Также происходит и с пользователями из USA, возможно они как раз и приходят из этого канала. Поэтому в первую очередь надо обратить внимание на данный рекламный канал и проверить ошибки.

Удержание платящих пользователе из FaceBoom много ниже, также стоит обратить внимание на этот канал.



Вообще по всем исследованиям видно, что чем больше вкладывались в рекламу - тем хуже была динамика ROI пользователе на 14-й день.


# Вывод:

Я бы посоветовала отделу маркетинга обратить большее внимание на пользователей из Германии, хоть оттуда приходит наименьшее число пользователей, зато платящих - достаточно большой процент. И обратить внимание на канал TipTop, так как стоимость рекламы увеличивается, а окупаемость нет. И удержание пользователей из канала FaceBoom низкое, по сравнению  с остальными каналами. Хоть платящие пользователи со всех устройств удерживаются практически одинаково, все же стоит проверить ошибки работы приложения на устроствах iPhone и Macю (стоимость привлечения пользователей MAC и iPhone увеличивается, а вот окупаемости нет.)

Неплохие показатели у каналов lambdaMediaAds и YRabbit, возможно стоит больше денег на рекламу вложить именно в них.